In [1]:
# from https://stackoverflow.com/questions/63594323/astropy-wcs-transfromation-matrix
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS
from astropy.wcs.utils import fit_wcs_from_points
import numpy as np
from astropy.coordinates.angle_utilities import angular_separation


In [2]:

# I have the following stars identified in my image (andromeda):
#  (X, Y)       --> (RA in degrees, DEC in degrees) --> HIP_ID
#  (640, 555)   --> (17.43421495, 35.61993419)      --> 5447
#  (1076, 32)  --> (2.09777329, 29.08952671)        --> 607
#  (161, 903)  --> (30.9751282, 42.32944223)        --> 9640
#  (932, 327)  --> (9.83272908, 30.86056254)        --> 3092
stars = SkyCoord(ra=[17.43421495, 2.09777329, 30.9751282, 9.83272908], 
                 dec=[35.61993419, 29.08952671, 42.32944223, 30.86056254], 
                 unit=u.deg)
stars
pixels_x = np.array([640, 1076, 161, 932])
pixels_y = np.array([555, 32, 903, 327])
wcs = fit_wcs_from_points((pixels_x, pixels_y), stars); wcs
wcs.wcs_pix2world(np.array([640]),np.array([555]),0, ra_dec_order=True)

(array([17.40217445]), array([35.59826335]))

In [3]:
ra_fit, dec_fit = wcs.wcs_pix2world(pixels_x,pixels_y,0, ra_dec_order=True)

In [4]:
resids = angular_separation(np.radians(ra_fit), np.radians(dec_fit), np.radians(stars.ra.deg), np.radians(stars.dec.deg))

In [5]:
np.degrees(resids), np.degrees(np.std(resids))

(array([0.03388489, 0.00190631, 0.01302561, 0.02369304]), 0.011920851463217357)

In [6]:
from astropy.io import fits
def test_fit_wcs_from_points():
    header_str_linear = """
XTENSION= 'IMAGE   '           / Image extension
BITPIX  =                  -32 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                   50
NAXIS2  =                   50
PCOUNT  =                    0 / number of parameters
GCOUNT  =                    1 / number of groups
RADESYS = 'ICRS    '
EQUINOX =               2000.0
WCSAXES =                    2
CTYPE1  = 'RA---TAN'
CTYPE2  = 'DEC--TAN'
CRVAL1  =    250.3497414839765
CRVAL2  =    2.280925599609063
CRPIX1  =               1045.0
CRPIX2  =               1001.0
CD1_1   =   -0.005564478186178
CD1_2   =   -0.001042099258152
CD2_1   =     0.00118144146585
CD2_2   =   -0.005590816683583
"""

    header_str_sip = """
XTENSION= 'IMAGE   '           / Image extension
BITPIX  =                  -32 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                   50
NAXIS2  =                   50
PCOUNT  =                    0 / number of parameters
GCOUNT  =                    1 / number of groups
RADESYS = 'ICRS    '
EQUINOX =               2000.0
WCSAXES =                    2
CTYPE1  = 'RA---TAN-SIP'
CTYPE2  = 'DEC--TAN-SIP'
CRVAL1  =    250.3497414839765
CRVAL2  =    2.280925599609063
CRPIX1  =               1045.0
CRPIX2  =               1001.0
CD1_1   =   -0.005564478186178
CD1_2   =   -0.001042099258152
CD2_1   =     0.00118144146585
CD2_2   =   -0.005590816683583
A_ORDER =                    2
B_ORDER =                    2
A_2_0   =    2.02451189234E-05
A_0_2   =   3.317603337918E-06
A_1_1   = 1.73456334971071E-05
B_2_0   =   3.331330003472E-06
B_0_2   = 2.04247482482589E-05
B_1_1   = 1.71476710804143E-05
AP_ORDER=                    2
BP_ORDER=                    2
AP_1_0  = 0.000904700296389636
AP_0_1  = 0.000627660715584716
AP_2_0  =  -2.023482905861E-05
AP_0_2  =  -3.332285841011E-06
AP_1_1  =  -1.731636633824E-05
BP_1_0  = 0.000627960882053211
BP_0_1  = 0.000911222886084808
BP_2_0  =  -3.343918167224E-06
BP_0_2  =  -2.041598249021E-05
BP_1_1  =  -1.711876336719E-05
A_DMAX  =    44.72893589844534
B_DMAX  =    44.62692873032506
"""
    header_linear = fits.Header.fromstring(header_str_linear, sep='\n')
    header_sip = fits.Header.fromstring(header_str_sip, sep='\n')

    true_wcs_linear = WCS(header_linear, relax=True)
    true_wcs_sip = WCS(header_sip, relax=True)

    # Getting the pixel coordinates
    x, y = np.meshgrid(list(range(10)), list(range(10)))
    x = x.flatten()
    y = y.flatten()

    # Calculating the true sky positions
    world_pix_linear = true_wcs_linear.pixel_to_world(x, y)
    world_pix_sip = true_wcs_sip.pixel_to_world(x, y)

    
    # Fitting the wcs, no distortion.
    fit_wcs_linear = fit_wcs_from_points((x, y), world_pix_linear,
                                         proj_point='center', sip_degree=None)

    # Fitting the wcs, with distortion.
    fit_wcs_sip = fit_wcs_from_points((x, y), world_pix_sip,
                                      proj_point='center', sip_degree=2)

    # Validate that the true sky coordinates calculated with `true_wcs_linear`
    # match sky coordinates calculated from the wcs fit with only linear terms

    world_pix_linear_new = fit_wcs_linear.pixel_to_world(x, y)

    dists = world_pix_linear.separation(world_pix_linear_new)
    
    assert dists.max() < 7e-5*u.deg
    assert np.std(dists) < 2.5e-5*u.deg

    # Validate that the true sky coordinates calculated with `true_wcs_sip`
    # match the sky coordinates calculated from the wcs fit with SIP of same
    # degree (2)

    world_pix_sip_new = fit_wcs_sip.pixel_to_world(x, y)
    dists = world_pix_sip.separation(world_pix_sip_new)

    assert dists.max() < 7e-6*u.deg
    assert np.std(dists) < 2.5e-6*u.deg

    # Test 360->0 degree crossover
    header_linear["CRVAL1"] = 352.3497414839765
    header_sip["CRVAL1"] = 352.3497414839765

    true_wcs_linear = WCS(header_linear, relax=True)
    true_wcs_sip = WCS(header_sip, relax=True)

    # Calculating the true sky positions
    world_pix_linear = true_wcs_linear.pixel_to_world(x, y)
    world_pix_sip = true_wcs_sip.pixel_to_world(x, y)

    # Fitting the wcs, no distortion.
    fit_wcs_linear = fit_wcs_from_points((x, y), world_pix_linear,
                                         proj_point='center', sip_degree=None)

    # Fitting the wcs, with distortion.
    fit_wcs_sip = fit_wcs_from_points((x, y), world_pix_sip,
                                      proj_point='center', sip_degree=2)

    # Validate that the true sky coordinates calculated with `true_wcs_linear`
    # match sky coordinates calculated from the wcs fit with only linear terms

    world_pix_linear_new = fit_wcs_linear.pixel_to_world(x, y)

    dists = world_pix_linear.separation(world_pix_linear_new)
    assert dists.max() < 7e-5*u.deg
    assert np.std(dists) < 2.5e-5*u.deg


In [7]:
test_fit_wcs_from_points()

In [8]:
import astropy
astropy.__path__

['/Users/yoachim/anaconda3/lib/python3.7/site-packages/astropy']